<a href="https://colab.research.google.com/github/LC1332/personality-text-generation/blob/main/lulu_exp/%E5%BF%83%E7%90%86%E9%97%AE%E7%AD%94%E5%AE%A1%E6%A0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
long_name = {
    'N':'neuroticism',
    'E':'extraversion',
    'O':'openness',
    'A':'agreeableness',
    'C':'conscientiousness'
}

input_name = 'questions.jsonl'

这个文件每行是一个json格式

请为我读取到python中，存到一个叫做datas的list中

In [7]:
input_name = 'questions.jsonl'

import json

datas = []

with open(input_name, encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        datas.append(data)

In [9]:
print(datas[1])

{'id': 1, 'question': '我真的喜欢大部份我遇见的人。', 'factor': 'extraversion', 'if_pos': 'true'}


In [12]:
!pip install -q langchain tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00


In [17]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [10]:
#@title 大五人格的分factor细节定义

# big5_to_prob_id = {
#     'neuroticism': [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56],
#     'extraversion': [2, 7, 12, 18, 22, 27, 32, 37, 42, 47, 52, 57],
#     'openness': [3, 8, 13, 17, 23, 28, 33, 38, 43, 48, 53, 58],
#     'agreeableness': [4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59],
#     'conscientiousness': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
# }

big5_to_detail = {}

big5_to_detail['conscientiousness'] = """Conscientiousness refers to the way we control, regulate, and direct our impulses. It assesses organization, persistence, and motivation in goal-directed behavior. It contrasts dependable, disciplined individuals with those who are lackadaisical and disorganized. Conscientiousness also reflects the level of self-control and the ability to delay gratification. Impulsiveness is not necessarily bad, sometimes the environment requires quick decision-making. Impulsive individuals are often seen as fun, interesting companions. However, impulsive behavior often gets people into trouble, providing momentary gratification at the expense of long-term negative consequences, such as aggression or substance abuse. Impulsive people generally do not accomplish major achievements. Conscientious people more easily avoid trouble and achieve greater success. They are generally seen as intelligent and reliable, although highly conscientious people may be perfectionists or workaholics. Extremely prudent individuals can seem monotonous, dull, and lifeless.

Conscientiousness can be divided into six facets:

C1 COMPETENCE

Refers to the sense that one is capable, sensible, prudent, and effective. High scorers feel well-prepared to deal with life. Low scorers have a lower opinion of their abilities, admitting that they are often unprepared and inept.

High scorers: Confident in own abilities. Efficient, thorough, confident, intelligent.

Low scorers: Lack confidence in own abilities, do not feel in control of work and life. Confused, forgetful, foolish.

C2 ORDER

High scorers are neat, tidy, well-organized, they put things in their proper places. Low scorers cannot organize things well, describe themselves as unmethodical.

High scorers: Well-organized, like making plans and following them. Precise, efficient, methodical.

Low scorers: Lack planning and orderliness, appear haphazard. Disorderly, impulsive, careless.

C3 DUTIFULNESS

To some extent, dutifulness refers to adherence to one's conscience, assessed by this facet. High scorers strictly follow their moral principles and scrupulously fulfill their moral obligations. Low scorers are more casual about such matters, somewhat unreliable or undependable.

High scorers: Dutiful, follow the rules. Reliable, polite, organized, thorough.

Low scorers: Feel restricted by rules and regulations. Often seen by others as unreliable, irresponsible. Careless, thoughtless, distracted.

C4 ACHIEVEMENT STRIVING

High scorers have high aspiration levels and work hard to achieve their goals. They are industrious, purposeful, and have a sense of direction. Low scorers are lackadaisical, even lazy, lacking motivation to succeed, having no ambitions and appearing to drift aimlessly. But they are often quite satisfied with their modest level of accomplishment.

High scorers: Striving for success and excellence, often seen as workaholics. Ambitious, industrious, enterprising, persevering.

Low scorers: Satisfied with completing basic tasks, seen as lazy by others. Leisurely, daydreaming, aimless.

C5 SELF-DISCIPLINE

Refers to the ability to begin tasks and follow through despite boredom or distractions. High scorers can motivate themselves to get the job done. Low scorers procrastinate in starting routine chores, easily losing confidence and giving up.

Low scorers: Procrastinate in work, often do not finish tasks, easily discouraged by obstacles. Unambitious, forgetful, distracted.

C6 DELIBERATION

This facet pertains to the tendency to think carefully before acting. High scorers are cautious and deliberate. Low scorers are hasty and speak/act without considering consequences.

High scorers: Think before acting, not impulsive. Prudent, logical, mature.

Low scorers: Act without considering consequences, impulsive, speak thoughts as they occur. Immature, rash, impulsive, careless.
"""

big5_to_detail['openness'] = """Openness describes a person's cognitive style. Openness to experience is defined as: the proactive seeking and appreciation of experience for its own sake, and tolerance for and exploration of the unfamiliar. This dimension contrasts intellectually curious, creative people open to novelty with traditional, down-to-earth, closed-minded individuals lacking artistic interests. Open people prefer abstract thinking, have wide interests. Closed people emphasize the concrete, conventional, are more traditional and conservative. Open people are suited to professions like teaching, closed people to occupations like police, sales, service.

Openness can be divided into six facets:

O1 FANTASY

Open people have vivid imaginations and active fantasy lives. Their daydreams are not just escapes, but ways to create interesting inner worlds. They elaborate and flesh out their fantasies, and believe imagination is essential for a rich, creative life. Low scorers are more prosaic, keeping their minds on the task at hand.

High scorers: Find the real world too plain and ordinary. Enjoy imagining, creating a more interesting, enriching world. Imaginative, daydreaming.

Low scorers: Matter-of-fact, prefers real-world thinking. Practical, prefer concrete thought.

O2 AESTHETICS

High scorers have deep appreciation for art and beauty. They are moved by poetry, absorbed in music, and touched by art. They may not have artistic talent or refined taste, but most have strong interests that enrich their experience. Low scorers are relatively insensitive and indifferent to art and beauty.

High scorers: Appreciate beauty in nature and the arts. Value aesthetic experiences, touched by art and beauty.

Low scorers: Insensitive to beauty, disinterested in the arts. Insensitive to art, cannot understand it.

O3 FEELINGS

Refers to receptivity to one's own inner feelings, evaluating emotions as an important part of life. High scorers experience deeper emotional states and can differentiate among them, experiencing happiness and unhappiness more intensely than others. Low scorers have blunted affect and do not believe feelings are important.

High scorers: Aware of own emotions/inner life. Sensitive, empathic, give importance to own feelings.

Low scorers: Little access to own emotions/inner world, unwilling to articulate them. Narrow range of emotions, insensitive to context.

O4 ACTIONS

Openness is seen behaviorally as willingness to try different activities, visit new places, or sample exotic foods. High scorers prefer novelty and variety to familiarity and routine. Over time, one high scorer may have a wide array of interests. Low scorers find change difficult and prefer to stick with known activities.

High scorers: Like experiencing new things, traveling, seeking different experiences. Find routine boring, willing to try new things. Seek novelty/variety, try new activities.

Low scorers: Uncomfortable with unfamiliar, prefer familiar surroundings/people. Set in ways, like familiar things.

O5 IDEAS

Intellectual curiosity is an aspect of openness, manifested both in pursuit of intellectual interests for their own sake and in open-mindedness to new, unconventional ideas. High scorers enjoy philosophical arguments and brain teasers. Low scorers have narrower intellectual interests, and even if they are intelligent concentrate their abilities within limited areas.

High scorers: Inquisitive, analytical, theoretical.

Low scorers: Pragmatic, fact-oriented, unappreciative of mental challenges.

O6 VALUES

Openness to values means constantly reexamining social, political, and religious values. Closed people tend to accept authority, honor tradition, and as a result are conservative regardless of political affiliation.

High scorers: Enjoy challenging authority, convention, traditional ideas. At the extreme, show hostility toward rules, sympathize with law-breakers. Tolerant, broad-minded, nonconforming.

Low scorers: Prefer the stability and security of authority and convention, unwilling to challenge the status quo. Dogmatic, conservative, conforming.
"""


big5_to_detail['agreeableness'] = """Agreeableness assesses the degree to which an individual is likable, while Agreeableness examines an individual's attitudes toward others, encompassing both a compassionate, sympathetic orientation along with antagonism, distrust, indifference. This facet represents the broad interpersonal orientation. Agreeableness represents "love", how much value is placed on cooperation and social harmony.

High scorers are good-natured, friendly, generous, helpful, willing to compromise their interests for others. They have an optimistic view of human nature, believing others to be honest, decent and trustworthy. Low scorers place self-interest above helping others. They are generally unconcerned with others' well-being and therefore unwilling to extend themselves for others. At times, they are overly suspicious of others' motives. For certain occupations, high agreeableness may not be optimal, especially when toughness and objective judgment is required, such as scientists, critics, and soldiers.

Agreeableness can be divided into six facets:

A1 TRUST

High scorers believe others are honest and well-intentioned. Low scorers tend to be cynical and suspicious, assuming others to be dishonest and dangerous.

High scorers: Believe others to be honest, reliable and well-meaning. Forgiving, trusting of others, good-natured.

Low scorers: View others as selfish, dangerous, looking to take advantage. Cautious, pessimistic, suspicious, hardhearted.

A2 STRAIGHTFORWARDNESS

High scorers are frank, sincere, ingenuous. Low scorers are more willing to manipulate others through flattery, craftiness, deception. They see it as an essential social skill and view straightforward people as naive.

High scorers: Believe there is no need for pretense in dealing with others, appear straightforward, genuine. Direct, frank, open, ingenuous.

Low scorers: Tend to be guarded in dealings with others, defensive, unwilling to reveal their full hand. Shrewd, slick, charming.

A3 ALTRUISM

High scorers actively concern themselves with others' welfare as evidenced by generosity, consideration of others, and a willingness to assist those in need. Low scorers are more self-centered and reluctant to get involved in others' problems.

High scorers: Willing to assist others, find helping others rewarding. Warm-hearted, soft-hearted, mild-mannered, generous, kindhearted.

Low scorers: Unwilling to help others, find helping a burden. Selfish, misanthropic, hardhearted, ungenerous.

A4 COMPLIANCE

Relates to personality in the context of interpersonal conflict. High scorers tend to defer to others, inhibit aggression, forgive and forget. Compliant people are gentle, mild-mannered. Low scorers are aggressive, preferring competition over cooperation, and unhesitatingly express anger when necessary.

High scorers: Dislike conflict with others, willing to give up their own standpoint or deny their own needs to get along. Deferent, accommodating, obliging.

Low scorers: Do not mind conflict with others, willing to intimidate others to achieve their goals. Stubborn, making unreasonable demands, obstinate, hardhearted.

A5 MODESTY

High scorers are self-effacing, humble. Low scorers believe they are superior and may be seen by others as arrogant, egotistical.

High scorers: Modest, unassuming.

Low scorers: Assertive, arrogant, vain, rude.

A6 TENDER-MINDEDNESS

Measures attitudes of sympathy and concern for others. High scorers are moved by others' needs and advocate humane social policies. Low scorers are hardheaded, unmoved by appeals to pity. They pride themselves on making objective appraisals based on cool logic.

High scorers: Sympathetic, moved by others' suffering, express pity. Friendly, warm-hearted, gentle, soft-hearted.

Low scorers: Do not strongly feel others' pain, pride themselves on objectivity, more concerned with truth and fairness than mercy. Callous, hardhearted, opinionated, ungenerous.
"""

big5_to_detail['extraversion'] = """Extraversion represents the quantity and intensity of interpersonal interaction, the need for stimulation, and capacity for joy. This dimension contrasts social, outgoing, action-oriented individuals with reserved, sober, shy, silent types. This trait can be measured through two facets: the level of interpersonal involvement and the activity level. The former evaluates the degree to which an individual enjoys the company of others. The latter reflects an individual's personal pace and vigor.

Extraverted people enjoy interacting with others, are full of energy, and often experience positive emotions. They are enthusiastic, enjoy physical activities, and like excitement and adventure. In a group, they are very talkative, confident, and enjoy being the center of attention.

Introverted people are quieter, more cautious, and do not enjoy too much interaction with the outside world. Their lack of desire for interaction should not be confused with shyness or depression, it is simply because compared to extraverts, they do not need as much stimulation and prefer being alone. An introvert's tendencies are sometimes wrongly viewed as arrogance or unfriendliness, but they are often very kind people once you get to know them.

Extraversion can be divided into the following six facets:

E1 WARMTH

Most relevant to interpersonal intimacy. Warm people are affectionate and friendly. They genuinely like others and easily form close relationships. Low scorers are not necessarily hostile or lacking in compassion, but are more formal, reserved, and detached in their behavior.

High scorers: Warm people enjoy those around them and often express positive, friendly emotions towards others. They are good at making friends and forming intimate relationships. Sociable, talkative, affectionate.

Low scorers: Although not necessarily cold or unfriendly, they are often seen as distant by others.

E2 GREGARIOUSNESS

Refers to a preference for other people's company. Gregarious people enjoy the company of others and the more people the merrier. Low scorers tend to be loners, they do not seek out and even actively avoid social stimulation.

High scorers: Enjoy being with people, prefer lively, crowded settings. Outgoing, having many friends, seek social affiliations.

Low scorers: Avoid crowds, find them draining. Prefer having more time alone, having their own personal space. Avoid crowds, enjoy solitude.

E3 ASSERTIVENESS

High scorers are dominant, forceful, and socially ascendant. They speak without hesitation and often become group leaders. Low scorers prefer to stay quietly in the background and let others do the talking.

High scorers: Like occupying a position of social dominance, directing others, influencing others' behavior. Dominant, forceful, confident, decisive.

Low scorers: Talk little in groups, allow others to occupy the dominant role. Modest, shy, quiet, reserved.

E4 ACTIVITY

High scorers lead fast-paced, vigorous lives, are energetic, and have a need to keep busy. Low scorers are more leisurely and relaxed, but not necessarily lazy or sluggish.

High scorers: Fast-paced, busy in work and life. Appear energetic, enjoy participating in many activities. Energetic, fast-paced, vigorous.

Low scorers: Slow-paced, leisurely in work and life. Unhurried, deliberate, composed.

E5 EXCITEMENT SEEKING

High scorers crave excitement and stimulation, enjoy bright colors and noisy environments. Excitement seeking is akin to some aspects of sensation seeking. Low scorers have little need for thrills and prefer activities seen as dull by the former.

High scorers: Easily get bored without stimulation, enjoy loud music/noise, like adventure, seek thrills. Flashy, seek intense experiences, thrill-seeking.

Low scorers: Avoid noise and crowds, dislike risky ventures. Cautious, sedate, not interested in thrills.

E6 POSITIVE EMOTIONS

Reflects the tendency to experience positive emotions (joy, happiness, love, excitement).

High scorers: Easily experience various positive moods like joy, optimism, excitement. Cheerful, elated, optimistic.

Low scorers: Do not readily experience positive emotions, but does not mean they often feel negative emotions either. Low scorers are simply less prone to excitement. Unemotional, calm, serious.
"""

big5_to_detail['neuroticism'] = """Neuroticism or Emotional Stability: Having tendencies of anxiety, hostility, depression, self-consciousness, impulsiveness, vulnerability.

N1 ANXIETY

Anxious individuals tend to worry, fear, be easily concerned, tense, and oversensitive. Those who score high are more likely to have free-floating anxiety and apprehension. Those with low scores tend to be calm, relaxed. They do not constantly worry about things that might go wrong.

High scorers: Anxiety, easily feel danger and threats, tend to be tense, fearful, worried, uneasy.

Low scorers: Calm state of mind, relaxed, not easily scared, won't always worry about things that could go wrong, emotions are calm, relaxed, stable.

N2 ANGRY HOSTILITY

Reflects the tendency to experience anger and related states (e.g. frustration, bitterness). Measures the ease with which an individual experiences anger.

High scorers: Easily angered, resentful when feeling unfairly treated, irritable, angry, frustrated.

Low scorers: Not easily angered/provoked, friendly, even-tempered, not easily provoked to anger.

N3 DEPRESSION

Measures individual differences in the tendency to experience depressive affect. High scorers are prone to feelings of guilt, sadness, hopelessness, and loneliness. They are easily discouraged and often feel blue. Low scorers rarely experience these emotions.

High scorers: Despairing, guilty, gloomy, dejected. Prone to feeling sorrow, abandonment, discouraged. Prone to feelings of guilt, sadness, disappointment, and loneliness. Easily discouraged, often feeling down.

Low scorers: Not prone to feeling sad, rarely feels abandoned.

N4 SELF-CONSCIOUSNESS

Core is shyness and embarrassability. Such individuals feel uncomfortable in groups, are sensitive to ridicule, and prone to feelings of inferiority. Self-consciousness is similar to shyness and social anxiety. Low scorers are not necessarily well-mannered or skilled socially, they are simply less disrupted by awkward social situations.

High scorers: Too concerned with what others think, afraid of being laughed at, tend to feel shy, anxious, inferior, awkward in social situations.

Low scorers: Composed, confident in social situations, not easily made tense or shy.

N5 IMPULSIVENESS

Refers to control over cravings and urges. Individuals give in too easily to impulses and temptations to care for long-term consequences (e.g. for food, cigarettes, possessions), although they will regret their actions later. Low scorers are better able to resist temptation and have higher frustration tolerance.

High scorers: Cannot resist cravings when experiencing strong urges, tend to pursue short-term satisfaction without considering long-term consequences. Cannot resist temptations, rash, spiteful, self-centered.

Low scorers: Self-controlled, can resist temptation.

N6 VULNERABILITY

Refers to susceptibility to stress. High scorers have difficulty coping with stress, become dependent, hopeless, panicked when facing emergencies. Low scorers feel that they can handle difficult situations properly.

High scorers: Under stress, easily feel panic, confusion, helpless, cannot cope with stress.

Low scorers: Under stress, feel calm, confident. Resilient, clear-headed, brave."""


In [143]:
prefix_prompt = "你扮演一个资深的心理学家。"



suffix_prompt = """我已经设计了一些访谈的问题，请为我评估这些访谈问题中的每一个，在访谈中是否能够正确测试出被试对应人格特质的程度
以及假设一些被试的回答，在reason中用"如果被试回答xxx，那就意味着xxx"的语气进行评估。输出成jsonl格式，每个问题的评估包含
"problem","if_plausible","related_sub_factor","reason"四个字段。
- if_plausible输出True或者False
- related_sub_factor使用O1, C2这样的字母表示
输出为
{
  "problem": <repeat the problem I designed>
  "if_plausible": <true or false>,
  "related_sub_factor": <related_sub_factor,指出problem对应能够测试被试的哪个子维度的人格特质>,
  "reason": <summarize the reason>
}
这样的格式:

"""



In [144]:
import os

key = "sk-OSqRHs2"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [145]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [146]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0.0)

In [147]:
smart_system_prompt = """You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2021-09
Current date: 2023-03-15"""

In [148]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

In [149]:
whole_message = prefix_prompt

problems = [x for x in datas if x['factor'] == current_factor]

current_factor = long_name['O']

target_message = f"我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 {current_factor}的程度"

whole_message += '\n'
whole_message += target_message
whole_message += '\n'

whole_message += big5_to_detail[current_factor]

whole_message += suffix_prompt

whole_message += "我设计的问题如下:\n"

for prob_id in range(4):
    whole_message += '- ' + problems[prob_id]['question']
    whole_message += '\n'


messages = [
    SystemMessage(content=smart_system_prompt),
    HumanMessage(content=whole_message),
]

In [150]:
response = chat(messages)
print(response.content)

{
  "problem": "我不喜欢浪费时间去做白日梦。",
  "if_plausible": false,
  "related_sub_factor": "O1",
  "reason": "如果被试回答不喜欢浪费时间去做白日梦，那就意味着他们在O1（FANTASY）子维度上得分较低。高分者通常有丰富的想象力和活跃的幻想生活，而低分者则更注重现实和实际思维。"
}

{
  "problem": "大自然和艺术的规律形态使我感到极为奥妙。",
  "if_plausible": true,
  "related_sub_factor": "O2",
  "reason": "如果被试回答大自然和艺术的规律形态使我感到极为奥妙，那就意味着他们在O2（AESTHETICS）子维度上得分较高。高分者通常对艺术和美有深刻的欣赏，被艺术和美所感动。低分者则相对不敏感和漠不关心艺术和美。"
}

{
  "problem": "我对诗词只有少许感觉甚至无动於衷。",
  "if_plausible": false,
  "related_sub_factor": "O2",
  "reason": "如果被试回答对诗词只有少许感觉甚至无动於衷，那就意味着他们在O2（AESTHETICS）子维度上得分较低。高分者通常对艺术和美有深刻的欣赏，被艺术和美所感动。低分者则相对不敏感和漠不关心艺术和美。"
}

{
  "problem": "当我阅读一首诗或欣赏一件艺术品时，我有时会感到兴奋或惊喜。",
  "if_plausible": true,
  "related_sub_factor": "O2",
  "reason": "如果被试回答当我阅读一首诗或欣赏一件艺术品时，我有时会感到兴奋或惊喜，那就意味着他们在O2（AESTHETICS）子维度上得分较高。高分者通常对艺术和美有深刻的欣赏，被艺术和美所感动。低分者则相对不敏感和漠不关心艺术和美。"
}


我现在有一个字符串

response = """

noise

{
  "if_plausible": true,
  "related_sub_factor": "O1",
  "reason": "If the participant agrees with the statement and expresses a dislike for daydreaming, it suggests a lower level of openness to fantasy (O1)."
}

{
  "if_plausible": true,
  "related_sub_factor": "O2",
  "reason": "If the participant agrees with the statement and expresses a deep appreciation for the beauty and mystery of nature and art, it suggests a higher level of openness to aesthetics (O2)."
}

noise string
"""

我想提取其中的json格式的信息，请为我实现一个python函数，输入为这样的字符串，输出为list of json

In [151]:
import json
import re

def extract_json(text):
    json_strings = re.findall(r'{.*?}', text, re.DOTALL)
    json_data = []
    for json_str in json_strings:
        json_data.append(json.loads(json_str))

    if len(json_data) != 0:
        return json_data

    # pattern = r"\n- if_plausible:\s*(.*?)\s*- related_sub_factor:\s*(.*?)\s*- reason:\s*(.*?)\s*"
    # matches = re.findall(pattern, response, re.DOTALL)

    # json_data = []
    # for mm in matches:
    #     plausible, sub_factor, reason = mm
    #     question_info = {
    #         "if_plausible": plausible,
    #         "related_sub_factor": sub_factor,
    #         "reason": reason
    #     }
    #     json_data.append(question_id)

    return json_data

# print(extract_json(response.content))

我希望从类似以下的文本中

批量提取信息成为jsonl，分别提取if_plausible, related_sub_factor和reason三个字段

response = """
作为资深心理学家，我将为您评估您设计的访谈问题，以确定它们是否能够正确测试被试的神经质程度。以下是每个问题的评估结果：

问题1: "我不是一个充满烦恼的人。"
- if_plausible: True
- related_sub_factor: N1
- reason: 如果被试回答"True"，那就意味着他们在神经质因素N1（焦虑）方面得分较低。他们可能更倾向于保持冷静和放松，不经常为可能出错的事情担心。

问题2: "我很少感到恐惧及焦虑。"
- if_plausible: True
- related_sub_factor: N1
- reason: 如果被试回答"True"，那就意味着他们在神经质因素N1（焦虑）方面得分较低。他们可能不容易感到恐惧、担心或焦虑，情绪相对较为稳定。
"""

请为我用python实现


In [152]:
# prob_in_chunk = []
# response_in_chunk = []

In [153]:
from tqdm import tqdm

count = 0

for short_name in long_name.keys():

    current_factor = long_name[short_name]

    problems = [x for x in datas if x['factor'] == current_factor]

    for i in tqdm(range(0, len(problems), 4)):
        selected_probs = problems[i:i+4]

        whole_message = prefix_prompt
        target_message = f"我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 {current_factor}的程度"

        whole_message += '\n'
        whole_message += target_message
        whole_message += '\n'

        whole_message += big5_to_detail[current_factor]

        whole_message += suffix_prompt

        whole_message += "我设计的问题如下:\n"

        #TODO : 改为从problems中4个一组抽取
        for prob in selected_probs:
            whole_message += '- ' + prob['question']
            whole_message += '\n'

        count = count + 1

        if count <= len(prob_in_chunk):
            continue

        prob_in_chunk.append(selected_probs)


        messages = [
            SystemMessage(content=smart_system_prompt),
            HumanMessage(content=whole_message),
        ]

        response = chat(messages)
        response_in_chunk.append(response.content)



100%|██████████| 6/6 [03:07<00:00, 31.27s/it]


In [178]:
# prob_in_chunk = []
# response_in_chunk = []

print(len(prob_in_chunk))
print(len(response_in_chunk))

29
28


In [185]:
print(prob_in_chunk[-28])
print(response_in_chunk[-28])

[{'id': 20, 'question': '我很少感到寂寞或忧郁。', 'factor': 'neuroticism', 'if_pos': 'false'}, {'id': 25, 'question': '有时我感到自已完全一文不值。', 'factor': 'neuroticism', 'if_pos': 'true'}, {'id': 30, 'question': '我很少感到忧郁或沮丧。', 'factor': 'neuroticism', 'if_pos': 'false'}, {'id': 35, 'question': '很多时候，当事情不对劲时，我会感到挫败及想放弃。', 'factor': 'neuroticism', 'if_pos': 'true'}]
{
  "problem": "我很少感到寂寞或忧郁。",
  "if_plausible": true,
  "related_sub_factor": "N3",
  "reason": "如果被试回答很少感到寂寞或忧郁，那就意味着他们在N3（Depression）这个人格特质上得分较低，不容易体验到内疚、悲伤、绝望和孤独等情绪。"
}

{
  "problem": "有时我感到自己完全一文不值。",
  "if_plausible": true,
  "related_sub_factor": "N3",
  "reason": "如果被试回答有时感到自己完全一文不值，那就意味着他们在N3（Depression）这个人格特质上得分较高，容易体验到自责、悲伤、绝望和孤独等情绪，感觉自己没有价值。"
}

{
  "problem": "我很少感到忧郁或沮丧。",
  "if_plausible": true,
  "related_sub_factor": "N3",
  "reason": "如果被试回答很少感到忧郁或沮丧，那就意味着他们在N3（Depression）这个人格特质上得分较低，不容易体验到悲伤、绝望和沮丧等情绪。"
}

{
  "problem": "很多时候，当事情不对劲时，我会感到挫败及想放弃。",
  "if_plausible": true,
  "related_sub_factor": "N3",
  "reason": "如果被试回答很多时候，当事

In [202]:
count = 0

# 这里不知道为啥被我整错位了

# for probs,response in zip(prob_in_chunk,response_in_chunk):

for i in range(len(response_in_chunk)):
    response = response_in_chunk[i]
    probs = prob_in_chunk[i+1]
    if len(probs) == len(extract_json(response)):
        print(count , ' ok', len(probs))
    else:
        print(count , ' not ok', len(probs), len(extract_json(response)))
    count += 1
        # print(response)
    # print()

0  ok 4
1  ok 4
2  ok 4
3  ok 4
4  ok 1
5  ok 4
6  ok 4
7  ok 4
8  ok 4
9  ok 4
10  ok 4
11  ok 4
12  ok 4
13  ok 4
14  ok 4
15  ok 1
16  ok 4
17  ok 4
18  ok 4
19  ok 4
20  ok 4
21  ok 4
22  ok 4
23  ok 4
24  ok 4
25  ok 4
26  ok 4
27  ok 4


In [204]:
count = 0

for i in range(len(response_in_chunk)):
    response = response_in_chunk[i]
    probs = prob_in_chunk[i+1]
    if len(probs) == len(extract_json(response)):
        # print(count , ' ok', len(probs))
        count += len(probs)
    else:
        pass

print(count)
print(len(datas))


106
110


In [211]:
import json

save_name = '/content/questions_verified.jsonl'

with open(save_name, 'w', encoding='utf-8') as f:
    for i in range(len(response_in_chunk)):
        response = response_in_chunk[i]
        probs = prob_in_chunk[i+1]
        if len(probs) == len(extract_json(response)):
            for prob, res in zip(probs,extract_json(response)):
                # print(prob)
                # print(res)

                data = {
                    'id': prob['id'],
                    'question': res['problem'],
                    'factor': prob['factor'],
                    'sub_factor': res['related_sub_factor'],
                    'if_pos':prob['if_pos'],
                    'if_plausible':res['if_plausible'],
                    'reason': res['reason'],
                    'prob_repeat':prob['question']
                }

                # datas.append(data)

                json.dump(data, f, ensure_ascii=False)
                f.write('\n')
        else:
            print(' not ok', len(probs), len(extract_json(response)))
        # break


In [187]:
sel_id = 4
print(prob_in_chunk[sel_id+1])
print(response_in_chunk[sel_id])

[{'id': 77, 'question': '我也会有自卑的感觉。你是否也会感到自己不如别人?这种比较会让你有什么样的感受?', 'factor': 'neuroticism', 'if_pos': 'synthesis'}]

评估结果:
{
  "problem": "你平时是如何制定和实现目标的呢?有没有什么特别的方法可以提高目标的完成度呢?",
  "if_plausible": true,
  "related_sub_factor": "C4",
  "reason": "这个问题涉及到被试在目标设定和实现方面的能力。如果被试提到他们制定明确的目标，并且有一套系统的方法来提高目标的完成度，那么可以推断被试在conscientiousness的facet C4（Achievement Striving）上得分较高。"
}

问题2:
- 你对追求卓越有什么独特的见解吗?怎样才能在生活中更进一步呢?

评估结果:
{
  "problem": "你对追求卓越有什么独特的见解吗?怎样才能在生活中更进一步呢?",
  "if_plausible": true,
  "related_sub_factor": "C1",
  "reason": "这个问题涉及到被试对追求卓越的态度和方法。如果被试表达了对追求卓越的重视，并提供了一些具体的方法或见解来在生活中取得更进一步，那么可以推断被试在conscientiousness的facet C1（Competence）上得分较高。"
}

问题3:
- 我有时挺拖延的。你平时如何规划时间、提高执行效率的呢?有什么技巧可以帮助自己更准时完成任务吗?

评估结果:
{
  "problem": "我有时挺拖延的。你平时如何规划时间、提高执行效率的呢?有什么技巧可以帮助自己更准时完成任务吗?",
  "if_plausible": true,
  "related_sub_factor": "C5",
  "reason": "这个问题涉及到被试在时间管理和执行效率方面的能力。如果被试提供了一些规划时间和提高执行效率的技巧，并强调准时完成任务的重要性，那么可以推断被试在conscientiousness的facet C5（Self-Discipline）上得分较高。"
}

问题4:
- 我开始工作时也常需要先进入状态。

In [188]:

suffix_prompt = """我已经设计了一些访谈的问题，请为我评估这些访谈问题中的每一个，在访谈中是否能够正确测试出被试对应人格特质的程度
以及假设一些被试的回答，在reason中用"如果被试回答xxx，那就意味着xxx"的语气进行评估。输出成jsonl格式，每个问题的评估包含
"problem","if_plausible","related_sub_factor","reason"四个字段。
- if_plausible输出True或者False
- related_sub_factor 指出problem对应能够测试被试的哪个子维度的人格特质
"""


In [ ]:
sel_id = 4
print(prob_in_chunk[sel_id+1])
print(response_in_chunk[sel_id])

In [194]:
sel_id = 4
# print(prob_in_chunk[sel_id+1])
# print(response_in_chunk[sel_id])

whole_message = prefix_prompt

# problems = [x for x in datas if x['factor'] == current_factor]

current_factor = prob_in_chunk[sel_id+1][0]['factor']

target_message = f"我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 {current_factor}的程度"

whole_message += '\n'
whole_message += target_message
whole_message += '\n'

whole_message += big5_to_detail[current_factor]

whole_message += suffix_prompt

whole_message += "我设计的问题如下:\n"

for prob in prob_in_chunk[sel_id+1]:
    whole_message += '- ' + prob['question']
    whole_message += '\n'

# print(whole_message)
messages = [
    SystemMessage(content=smart_system_prompt),
    HumanMessage(content=whole_message),
]

response = chat(messages)
print(response.content)


{
  "problem": "我也会有自卑的感觉。你是否也会感到自己不如别人?这种比较会让你有什么样的感受?",
  "if_plausible": true,
  "related_sub_factor": "N4 SELF-CONSCIOUSNESS",
  "reason": "如果被试回答是肯定的，表示被试在自我意识方面得分较高，对自己与他人的比较敏感，容易感到自卑和不如别人。"
}


In [192]:
len(extract_json(response.content))

0

In [193]:
print(response.content)

根据你提供的问题，我将评估每个问题在访谈中是否能够正确测试出被试在大五人格特质中conscientiousness的程度。以下是对每个问题的评估结果：

问题1:
- 你平时是如何制定和实现目标的呢?有没有什么特别的方法可以提高目标的完成度呢?

评估结果:
- if_plausible: True
- related_sub_factor: C4 (Achievement Striving)
- reason: 如果被试回答他们制定明确的目标并采取积极的行动来实现这些目标，那就意味着他们在C4（追求成就）方面得分较高。这表明他们有较高的抱负水平，努力工作以实现自己的目标。

问题2:
- 你对追求卓越有什么独特的见解吗?怎样才能在生活中更进一步呢?

评估结果:
- if_plausible: True
- related_sub_factor: C4 (Achievement Striving)
- reason: 如果被试回答他们对追求卓越有强烈的渴望，并提出了具体的方法和策略来在生活中取得更大的进步，那就意味着他们在C4（追求成就）方面得分较高。这表明他们有较高的抱负水平，努力追求卓越。

问题3:
- 我有时挺拖延的。你平时如何规划时间、提高执行效率的呢?有什么技巧可以帮助自己更准时完成任务吗?

评估结果:
- if_plausible: True
- related_sub_factor: C5 (Self-Discipline)
- reason: 如果被试回答他们采取了明确的时间规划和执行效率提高的方法，并提供了一些技巧来帮助自己更好地管理时间和准时完成任务，那就意味着他们在C5（自律）方面得分较高。这表明他们有较强的自我控制能力和执行力。

问题4:
- 我开始工作时也常需要先进入状态。你是如何快速投入工作的呢?有什么诀窍可以帮助自己更高效地开始工作吗?

评估结果:
- if_plausible: True
- related_sub_factor: C6 (Deliberation)
- reason: 如果被试回答他们有一些方法和技巧可以帮助自己快速进入工作状态，并提到了一些策略来提高工作效率，那就意味着他们在C6（审慎性）方面得分较高。这表明他们在行动之前会仔细考虑，并能够迅速适应工作环境。

以上是对你设计的问题在评估被试conscie

In [195]:
response_in_chunk[4] = """
{
  "problem": "我也会有自卑的感觉。你是否也会感到自己不如别人?这种比较会让你有什么样的感受?",
  "if_plausible": true,
  "related_sub_factor": "N4 SELF-CONSCIOUSNESS",
  "reason": "如果被试回答是肯定的，表示被试在自我意识方面得分较高，对自己与他人的比较敏感，容易感到自卑和不如别人。"
}
"""

In [196]:
sel_id = 8

print(response_in_chunk[sel_id])

根据你提供的问题，我将评估每个问题在访谈中是否能够正确测试出被试对应人格特质的程度，并提供假设一些被试的回答的评估。以下是评估结果：

1. 问题: 我最近参加了一个聚会，认识了很多新朋友，感觉交谈的时候彼此距离拉近了很多。你平时喜欢参加聚会认识新朋友吗？
   - if_plausible: True
   - related_sub_factor: E2 Gregariousness
   - reason: 这个问题可以测试被试在E2 Gregariousness子维度中的程度。如果被试回答喜欢参加聚会认识新朋友，那就意味着他们在社交方面更加外向，喜欢与他人交往，享受与人相处的经历。

2. 问题: 我个人更喜欢一个人安静工作，可以让我更专注。你在工作的时候喜欢跟其他人一起合作，还是更习惯自己一个人独立完成呢？
   - if_plausible: True
   - related_sub_factor: E3 Assertiveness
   - reason: 这个问题可以测试被试在E3 Assertiveness子维度中的程度。如果被试回答更喜欢一个人安静工作，更习惯自己独立完成任务，那就意味着他们在社交方面相对较内向，更倾向于保持低调，不太喜欢在工作中与他人合作。

3. 问题: 我每天都保持很高的工作效率，生活也比较忙碌。你平时的生活节奏是较快还是较慢的呢？喜欢积极参与各种活动吗？
   - if_plausible: True
   - related_sub_factor: E4 Activity
   - reason: 这个问题可以测试被试在E4 Activity子维度中的程度。如果被试回答生活节奏较快，喜欢积极参与各种活动，那就意味着他们在生活中更加快节奏，喜欢保持忙碌和参与各种活动。

4. 问题: 你认为自己是一个乐观开朗的人吗？生活中容易感受到快乐吗？
   - if_plausible: True
   - related_sub_factor: E6 Positive Emotions
   - reason: 这个问题可以测试被试在E6 Positive Emotions子维度中的程度。如果被试回答认为自己是一个乐观开朗的人，生活中容易感受到快乐，那就意味着他们更容易体验到积极的情绪，如喜悦、乐观和兴奋。

请注意，以上评估结

In [197]:
response_in_chunk[8] = """
{
  "problem": "我最近参加了一个聚会，认识了很多新朋友，感觉交谈的时候彼此距离拉近了很多。你平时喜欢参加聚会认识新朋友吗？",
  "if_plausible": true,
  "related_sub_factor": "E2 Gregariousness",
  "reason": "这个问题可以测试被试在E2 Gregariousness子维度中的程度。如果被试回答喜欢参加聚会认识新朋友，那就意味着他们在社交方面更加外向，喜欢与他人交往，享受与人相处的经历。"
}
{
  "problem": "我个人更喜欢一个人安静工作，可以让我更专注。你在工作的时候喜欢跟其他人一起合作，还是更习惯自己一个人独立完成呢?",
  "if_plausible": true,
  "related_sub_factor": "E3 Assertiveness",
  "reason": "这个问题可以测试被试在E3 Assertiveness子维度中的程度。如果被试回答更喜欢一个人安静工作，更习惯自己独立完成任务，那就意味着他们在社交方面相对较内向，更倾向于保持低调，不太喜欢在工作中与他人合作。"
}
{
  "problem": "我每天都保持很高的工作效率，生活也比较忙碌。你平时的生活节奏是较快还是较慢的呢？喜欢积极参与各种活动吗?",
  "if_plausible": true,
  "related_sub_factor": "E4 Activity",
  "reason": "这个问题可以测试被试在E4 Activity子维度中的程度。如果被试回答生活节奏较快，喜欢积极参与各种活动，那就意味着他们在生活中更加快节奏，喜欢保持忙碌和参与各种活动。"
}
{
  "problem": "你认为自己是一个乐观开朗的人吗？生活中容易感受到快乐吗？",
  "if_plausible": true,
  "related_sub_factor": "E6 Positive Emotions",
  "reason": "这个问题可以测试被试在E6 Positive Emotions子维度中的程度。如果被试回答认为自己是一个乐观开朗的人，生活中容易感受到快乐，那就意味着他们更容易体验到积极的情绪，如喜悦、乐观和兴奋。"
}
"""

In [199]:
sel_id = 27

print(response_in_chunk[sel_id])

根据你提供的问题，我将评估每个问题在访谈中是否能够正确测试出被试在大五人格特质中conscientiousness的程度。以下是对每个问题的评估结果：

问题1:
- 你平时是如何制定和实现目标的呢?有没有什么特别的方法可以提高目标的完成度呢?

评估结果:
- if_plausible: True
- related_sub_factor: C4 (Achievement Striving)
- reason: 如果被试回答他们制定明确的目标并采取积极的行动来实现这些目标，那就意味着他们在C4（追求成就）方面得分较高。这表明他们有较高的抱负水平，并且努力工作以实现自己的目标。

问题2:
- 你对追求卓越有什么独特的见解吗?怎样才能在生活中更进一步呢?

评估结果:
- if_plausible: True
- related_sub_factor: C4 (Achievement Striving)
- reason: 如果被试回答他们对追求卓越有强烈的渴望，并提出了具体的方法和策略来在生活中取得更大的进步，那就意味着他们在C4（追求成就）方面得分较高。这表明他们有较高的抱负水平，并且致力于不断提高自己。

问题3:
- 我有时挺拖延的。你平时如何规划时间、提高执行效率的呢?有什么技巧可以帮助自己更准时完成任务吗?

评估结果:
- if_plausible: True
- related_sub_factor: C5 (Self-Discipline)
- reason: 如果被试回答他们采取了明确的时间规划和执行效率提高的方法，并提供了一些技巧来帮助自己更好地管理时间和准时完成任务，那就意味着他们在C5（自律性）方面得分较高。这表明他们有较强的自我约束能力和执行力。

问题4:
- 我开始工作时也常需要先进入状态。你是如何快速投入工作的呢?有什么诀窍可以帮助自己更高效地开始工作吗?

评估结果:
- if_plausible: True
- related_sub_factor: C6 (Deliberation)
- reason: 如果被试回答他们有一些方法和技巧可以帮助自己快速进入工作状态，并提到了一些策略来提高工作效率，那就意味着他们在C6（审慎性）方面得分较高。这表明他们在行动之前会仔细考虑，并具有较高的决策思考能力。

以上是对你设计的问题在评估

In [201]:
response_in_chunk[27] = """
{
  "problem": "你平时是如何制定和实现目标的呢?有没有什么特别的方法可以提高目标的完成度呢?",
  "if_plausible": true,
  "related_sub_factor": "C4 (Achievement Striving)",
  "reason": "如果被试回答他们制定明确的目标并采取积极的行动来实现这些目标，那就意味着他们在C4（追求成就）方面得分较高。这表明他们有较高的抱负水平，并且努力工作以实现自己的目标。"
}
{
  "problem": "你对追求卓越有什么独特的见解吗?怎样才能在生活中更进一步呢?",
  "if_plausible": true,
  "related_sub_factor": "C4 (Achievement Striving)",
  "reason": "如果被试回答他们对追求卓越有强烈的渴望，并提出了具体的方法和策略来在生活中取得更大的进步，那就意味着他们在C4（追求成就）方面得分较高。这表明他们有较高的抱负水平，并且致力于不断提高自己。"
}
{
  "problem": "我有时挺拖延的。你平时如何规划时间、提高执行效率的呢?有什么技巧可以帮助自己更准时完成任务吗?",
  "if_plausible": true,
  "related_sub_factor": "C5 (Self-Discipline)",
  "reason": "如果被试回答他们采取了明确的时间规划和执行效率提高的方法，并提供了一些技巧来帮助自己更好地管理时间和准时完成任务，那就意味着他们在C5（自律性）方面得分较高。这表明他们有较强的自我约束能力和执行力。"
}
{
  "problem": "我开始工作时也常需要先进入状态。你是如何快速投入工作的呢?有什么诀窍可以帮助自己更高效地开始工作吗?",
  "if_plausible": true,
  "related_sub_factor": "C6 (Deliberation)",
  "reason": "如果被试回答他们有一些方法和技巧可以帮助自己快速进入工作状态，并提到了一些策略来提高工作效率，那就意味着他们在C6（审慎性）方面得分较高。这表明他们在行动之前会仔细考虑，并具有较高的决策思考能力。"
}
"""

whole_message = prefix_prompt


current_factor = long_name['O']

problems = [x for x in datas if x['factor'] == current_factor]

##TODO 在这里外部增加一个for循环

target_message = f"我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 {current_factor}的程度"

whole_message += '\n'
whole_message += target_message
whole_message += '\n'

whole_message += big5_to_detail[current_factor]

whole_message += suffix_prompt

whole_message += "我设计的问题如下:\n"

#TODO : 改为从problems中4个一组抽取
for prob_id in range(4):
    whole_message += '- ' + problems[prob_id]['question']
    whole_message += '\n'


messages = [
    SystemMessage(content=smart_system_prompt),
    HumanMessage(content=whole_message),
]

response = chat(messages)

json_response = extract_json(response.content)

我想在这段代码的前面增加一个for循环，把中间的prob_id变为4个一组抽取（现在只抽取了前4个）。如果最后一组不满4个则有多少抽取多少。请为我用python实现